# Analyse et visualisation de données avec Python
## Collection de jeux de données Vega - Partie 2
Cette page contient des exercices basés sur les données de la collection Vega.


* Dépôt GitHub de [Vega Datasets](https://github.com/vega/vega-datasets).
  * [Origine](https://github.com/vega/vega-datasets/blob/master/SOURCES.md) des différents fichiers.
* Les exercices ci-dessous supposent que la copie locale partage le même dossier parent que `analyse-donnees-python`.

In [ ]:
import os
import pandas as pd

dossier_data = os.path.join(".." if os.path.basename(os.getcwd()) == "solutions" else ".",
                            "..", "..", "vega-datasets", "data")
print(dossier_data)

## Exercice 4 - Automatisation
Pour chaque capitale d'état aux États-Unis, trouvez l'aéroport le plus près, et ce, à l'aide des longitudes et des latitudes. Pour chaque aéroport trouvé, sauvegardez dans un fichier CSV les vols d'arrivée et de départ. Le nom de chaque fichier doit avoir le code de deux lettres de l'état dans lequel se situe l'aéroport, le nom de la capitale et le code IATA de l'aéroport, le tout séparé d'un trait d'union.

Par exemple : `XY-Nom Ville-ABC.csv`

Enfin, ces fichiers seront sauvegardés dans le dossier `aero_cap`.

a) Chargez les données et étudiez les différentes variables (colonnes).

In [ ]:
# Charger les données
capitales = pd.read_json(os.path.join(dossier_data, "us-state-capitals.json"))
aeroports = pd.read_csv(os.path.join(dossier_data, "airports.csv"))
vols      = pd.read_csv(os.path.join(dossier_data, "flights-3m.csv"))

In [ ]:
capitales.head()

In [ ]:
aeroports.head()

In [ ]:
vols.head()

b) Limitez les aéroports à ceux présents dans le DataFrame `vols`.

In [ ]:
# Recueillir les différents aéroports d'origine et de destination
orig = vols['origin'].unique()
dest = vols['destination'].unique()

# Ne garder que les aéroports d'origine et de destination
aeroports = aeroports[aeroports['iata'].isin(orig) |
                      aeroports['iata'].isin(dest)]

# Test unitaire : devrait retourner 59 aéroports
len(aeroports)

c) Créez le dossier `aero_cap` :
* Créez la fonction `creer_dossier()` qui reçoit un `nom_dossier` en argument.
* Validez que le dossier existe avant de le créer.
* Testez la fonction `creer_dossier()` à deux reprises

In [ ]:
def creer_dossier(nom_dossier):
    """Crée un dossier s'il n'existe pas
    
    nom_dossier -- Nom du dossier (str)
    """

    if nom_dossier in os.listdir('.'):
        print(f'Le dossier "{nom_dossier}" existe déjà!')
    else:
        os.mkdir(nom_dossier)
        print(f'Nouveau dossier: "{nom_dossier}"!')

In [ ]:
dossier_cible = "aero_cap"

creer_dossier(dossier_cible)
os.listdir('.')

d) Créez la fonction `dist_ang2(lat1, long1, lat2, long2)` qui reçoit deux paires de coordonnées et qui calcule le carré de la distance entre ces coordonnées.

In [ ]:
def dist_ang2(lat1, long1, lat2, long2):
    """Retourne le carré de la distance angulaire
    
    lat1, lat2 -- Latitudes (scalaire ou vecteur)
    long1, long2 -- Longitudes (scalaire ou vecteur)
    """
    
    # Différences calculées élément par élément
    delta_lat = lat2 - lat1
    delta_long = long2 - long1
    
    # Retourne le carré de la distance angulaire
    return delta_lat * delta_lat + delta_long * delta_long

In [ ]:
# Test unitaire : doit retourner 25 = (5-2)*(5-2)+(7-3)*(7-3)
dist_ang2(2, 3, 5, 7)

e) Créez la fonction `trouver_aeroport()` avec les arguments décrits dans le Docstring. Complétez la fonction en tenant compte des commentaires.

In [ ]:
def trouver_aeroport(aeroports, latitude, longitude):
    """Retourne le code IATA de l'aéroport le plus proche
    
    aeroports -- DataFrame des aeroports
    latitude -- degré de latitude
    longitude -- degré de longitude
    """
    
    # Copier les colonnes iata, latitude et longitude
    iata_long_lat = aeroports[['iata', 'latitude', 'longitude']].copy()
    
    # Appeler dist_ang2() avec les coordonnées des aéroports
    # et les coordonnées reçues en argument
    iata_long_lat['dist_ang2'] = dist_ang2(
        iata_long_lat['latitude'], iata_long_lat['longitude'], latitude, longitude)
    
    # Obtenir la distance minimale
    dist_ang2_min = iata_long_lat['dist_ang2'].min()
    
    # Garder la ou les rangées de cette distance minimale
    aero_locales = iata_long_lat[iata_long_lat['dist_ang2'] == dist_ang2_min]
    
    # Réinitialiser l'index et retourner le premier code IATA
    return aero_locales.reset_index().loc[0, 'iata']

In [ ]:
# Test unitaire : doit retourner 'OMA'
trouver_aeroport(aeroports, 40, -100)

f) Créez la fonction `trouver_code_etat()` avec les arguments décrits dans le Docstring. Complétez la fonction en tenant compte des commentaires.

In [ ]:
def trouver_code_etat(aeroports, code_iata):
    """Retourne le code d'état selon le code IATA
    
    aeroports -- DataFrame des aeroports
    code_iata -- Typiquement un code de 3 lettres
    """
    
    # Copier la rangée où le code IATA correspond
    aeroport = aeroports[aeroports['iata'] == code_iata].copy()
    
    # Réinitialiser l'index et retourner le premier code IATA
    return aeroport.reset_index().loc[0, 'state']

g) Complétez la boucle sur chaque `capitale` des états :
* L'itérable est donnée par la méthode [`df.itertuples()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.itertuples.html).
* Appelez la fonction `trouver_aeroport()` pour obtenir le code IATA.
* Appelez la fonction `trouver_code_etat()` pour obtenir le code d'état.
* Sélectionnez les `vols` dont l'origine **ou** la destination est le code IATA.
* Sauvegardez cette sélection dans le fichier CSV.

In [ ]:
# Pour chaque capitale
for capitale in capitales.itertuples(name="Capitale", index=False):
    # Obtenir le code IATA pour ensuite obtenir le code d'état
    code_iata = trouver_aeroport(aeroports, capitale.lat, capitale.lon)
    code_etat = trouver_code_etat(aeroports, code_iata)
    
    # Sélectionner les vols concernant l'aéroport choisi
    vols_cap = vols[(vols['origin'] == code_iata) |
                    (vols['destination'] == code_iata)]
    
    # Syntaxe du nom de fichier : "XY-Nom Ville-ABC.csv"
    nom_fichier = code_etat + "-" + capitale.city + "-" + code_iata + ".csv"
    
    # Sauvegarder vols_cap dans un fichier CSV dans dossier_cible
    vols_cap.to_csv(os.path.join(dossier_cible, nom_fichier))

os.listdir(dossier_cible)

## Exercice 5 - Visualisation
Dans cet exercice, nous voulons créer un graphique montrant l'évolution du S&P 500 au cours des mois recensés dans `sp500-2000.csv`. Plus précisément, on veut voir une courbe rouge pour les valeurs les plus basses à chaque mois et une autre courbe noire pour les valeurs hautes.

a) Importez le module Plotnine sous le nom `p9`.

In [ ]:
import plotnine as p9

b) Chargez les données du S&P 500 à partir du fichier `sp500-2000.csv`.

In [ ]:
sp500 = pd.read_csv(os.path.join(dossier_data, "sp500-2000.csv"))
sp500

c) Créez trois colonnes `annee`, `mois` et `jour` à partir de la colonne `date`.
* La méthode `.apply()` et la déclaration `lambda` permettent d'effectuer un traitement pour chaque valeur `d` de la colonne spécifiée (`'date'`).
* Note : `"AAAA-MM-JJ".split('-')` retourne une liste `['AAAA', 'MM', 'JJ']`.

In [ ]:
sp500['annee'] = sp500['date'].apply(lambda d : d.split('-')[0])
sp500['mois' ] = sp500['date'].apply(lambda d : d.split('-')[1])
sp500['jour' ] = sp500['date'].apply(lambda d : d.split('-')[2])
sp500

d) Pour chaque combinaison `annee,mois`, calculez la valeur minimale des valeurs `low` et la valeur maximale des valeurs `high`. Concaténez **horizontalement** ces deux résultats ; ici, c'est une opération pertinente, car les index sont les mêmes et les colonnes sont différentes.

In [ ]:
par_annee_mois = sp500.groupby(['annee', 'mois'])
minima = par_annee_mois['low' ].min()
maxima = par_annee_mois['high'].max()
extrema = pd.concat([minima, maxima], axis=1)
extrema

e) Pour ajuster le précédent résultat selon nos besoins :
* Renommez les deux colonnes restantes pour `Basses` et `Hautes`.
* Ramenez l'index `annee,mois` sous la forme de colonne avec `reset_index()`.

In [ ]:
extrema.columns = ['Basses', 'Hautes']
extrema = extrema.reset_index()
extrema

f) Créez une colonne `Mois` en type `datetime64`.

In [ ]:
extrema['Mois'] = extrema['annee'] + "-" + extrema['mois'] + "-01T00:00:00"
extrema['Mois'] = extrema['Mois'].astype("datetime64")
extrema

g) Utilisez la fonction [`pd.melt()`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) de telle sorte que :
* `id_vars` contienne une liste des colonnes à préserver (`Mois`).
* `value_vars` contienne une liste des colonnes à transformer en catégories (`Basses` et `Hautes`), ce qui fait que leurs valeurs se retrouveront dans une même colonne.
* `var_name` soit `Valeurs` comme nom de colonne pour les catégories `Basses` et `Hautes`.
* `value_name` soit `Valeur`, comme dans "valeur boursière".

In [ ]:
valeurs_finales = pd.melt(extrema, id_vars=['Mois'], value_vars=['Basses', 'Hautes'],
                          var_name='Valeurs', value_name='Valeur')
valeurs_finales

h) Enfin, créez un graphique `ggplot()` avec `p9` de telle sorte que :
* Les `Mois` soient en axe des `x`.
* Les `Valeur` soient en axe des `y`.
* Les `Valeurs` influencent la couleur (`color`).
* Une courbe s'affiche pour chaque type de `Valeurs`.
* Les couleurs `red` et `black` soit assignées via une liste dans `p9.scale_color_manual()`.
* Un titre soit ajouté avec `ggtitle()`.

In [ ]:
(p9.ggplot(data=valeurs_finales,
           mapping=p9.aes(x='Mois',
                          y='Valeur',
                          color='Valeurs'))
    + p9.geom_line()
    + p9.scale_color_manual(['red', 'black'])
    + p9.ggtitle("Valeurs basses et hautes du S&P 500 selon le mois")
)